In [2]:
import pandas as pd
# Load spreadsheet
xl = pd.ExcelFile('dataset.xlsx')
df = xl.parse('sheet 1')


In [5]:
df.columns=['video','time']

In [9]:
df=df.dropna()


In [12]:
import re
def rgx(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    return re.findall(pattern,url)[0]

df['id'] = df['video'].apply(rgx)


In [14]:
df.to_excel('new_dataset.xlsx', index=False)

In [3]:
import pandas as pd
df = pd.ExcelFile('new_dataset.xlsx')
df = df.parse('Sheet1')

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_subtitles(vid):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id=f'{vid}')
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

df['message_object']=df['id'].apply(get_subtitles)


An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=RpZNKZo0e1M! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=3xHrMwy-Y5A! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which versi

In [5]:
df['message_object']

0     [{'text': 'go to Google fonts if you want to',...
1     [{'text': 'hey guys welcome to research rocks ...
2     [{'text': 'so I have this sample web project h...
3     [{'text': 'the live share extension for vias c...
4     [{'text': 'hi there it's Peter here again the ...
5     [{'text': 'hey guys in this video I'm going to...
6     [{'text': 'hi welcome back so in this video I ...
7     [{'text': 'hey everyone so i've been gone a lo...
8     [{'text': 'are you interested in working with'...
9                                                  None
10    [{'text': 'not showing my audience how to buil...
11    [{'text': 'welcome everyone today i have anoth...
12    [{'text': 'hey there let's have a look at two ...
13    [{'text': 'hello guys welcome back to my chann...
14                                                 None
15                                                 None
16    [{'text': 'how can I connect to mongodb from v...
17    [{'text': 'hey guys in this video I'm goin

In [10]:
#now clear the rows who don't have message object
df.query(' message_object.isnull()')
df=df.dropna(subset=['message_object'])

In [13]:
df.to_excel('newest_dataset.xlsx',index=False)

Lets format the messages now


In [29]:
import pandas as pd
import ast
df=pd.ExcelFile('newest_dataset.xlsx').parse('Sheet1')

def safe_literal_eval(val):
    try:
        # Remove newline characters and extra spaces
        cleaned_val = val.replace('\n', ' ').replace('\r', '').strip()
        return ast.literal_eval(cleaned_val)
    except Exception as e:
        print(f"Error parsing the value: {val} - {e}")
        return []

# Apply the cleaning and evaluation function
df['message_object'] = df['message_object'].apply(safe_literal_eval)

def apply_subtitle(subtitles):
    news=''
    for subtitle in subtitles:
            news+=f" <start>  {subtitle['start']} - {subtitle['start'] + subtitle['duration']} : {subtitle['text']} "
    return news+'<start>'

df['message']=df['message_object'].apply(apply_subtitle)


Error parsing the value: [{'text': 'welcome everyone today i have another', 'start': 0.08, 'duration': 4.64}, {'text': 'exciting new video for you guys uh first', 'start': 2.08, 'duration': 5.12}, {'text': "of all my name is dave i'm a data", 'start': 4.72, 'duration': 5.36}, {'text': 'scientist and in this video i will show', 'start': 7.2, 'duration': 5.04}, {'text': 'you how to set up fies code for data', 'start': 10.08, 'duration': 3.76}, {'text': 'science this is something that has', 'start': 12.24, 'duration': 3.519}, {'text': 'completely changed the game for me', 'start': 13.84, 'duration': 3.68}, {'text': 'coming from working with jupiter', 'start': 15.759, 'duration': 4.241}, {'text': 'notebooks and jupiter lab which are', 'start': 17.52, 'duration': 4.72}, {'text': 'awesome tools by the way but vs code has', 'start': 20.0, 'duration': 3.92}, {'text': 'totally changed the game for me in terms', 'start': 22.24, 'duration': 4.0}, {'text': 'of my workflow and my productivity so in

In [35]:
df['message'][0]
df=df[df['message']!='<start>']

In [37]:
df.to_excel('final_data.xlsx',index=False)

In [39]:
df['message'][0]

" <start>  0.0 - 5.4 : go to Google fonts if you want to  <start>  2.879 - 7.5 : explore fonts filter them by Mono Styles  <start>  5.4 - 10.679 : or search for any font that you like  <start>  7.5 - 14.120000000000001 : I'll do space mono download the font  <start>  10.679 - 14.120000000000001 : then install it  <start>  17.46 - 23.939 : Now open up vs code settings from this  <start>  20.52 - 26.34 : gear icon search for font family and put  <start>  23.939 - 28.560000000000002 : your font name in front of others make  <start>  26.34 - 30.24 : sure that you spell it right check your  <start>  28.56 - 33.18 : code and it's got a new outfit now  <start>  30.24 - 35.399 : that's all thanks for watching support  <start>  33.18 - 39.38 : by subscribing to codemite and sharing  <start>  35.399 - 39.38 : the video okay like it too <start>"

let's make the jsonl file


In [2]:
import pandas as pd
df=pd.ExcelFile('final_data.xlsx')
import json

# Open and read the JSON file
with open('template.json', 'r') as file:
    template= json.load(file)


JSONDecodeError: Invalid control character at: line 6 column 360 (char 443)